Let's get to the point.
I'm refactoring old code to fit into the old models that I made
The problem is the preprocessing
And as always my crutch on AI has lead me astray
Regardless, I think I can get through this?

This notebook was written based on these resources:
https://www.kaggle.com/code/someadityamandal/bitcoin-time-series-forecasting/notebook
https://github.com/dataquestio/project-walkthroughs/tree/master/bitcoin_price
https://www.geeksforgeeks.org/time-series-forecasting-using-tensorflow/

Special thank you to David for, as always, having the best resources.

In [1]:
# Let's try all that preprocessing again.
import numpy as np
import pandas as pd
import keras.layers as layers
import keras.models as models

import datetime, pytz
from sklearn.preprocessing import MinMaxScaler

# import os
# <!-- print(os.listdir('./data')) -->

2024-11-16 00:42:14.322537: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 00:42:14.368849: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 00:42:14.869211: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 00:42:15.127194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 00:42:15.241511: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [6]:
# Okay so for some reasons those imports were taking WAY too long.

import yfinance as yf
import os

btc_ticker = yf.Ticker('BTC-USD')

if os.path.exists('btc.csv'):
    btc = pd.read_csv('btc.csv', index_col=0)
else:
    btc=btc_ticker.history(perid='max')
    btc.to_csv('btc.csv')

btc
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3713 entries, 2014-09-17 00:00:00+00:00 to 2024-11-16 00:00:00+00:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          3713 non-null   float64
 1   High          3713 non-null   float64
 2   Low           3713 non-null   float64
 3   Close         3713 non-null   float64
 4   Volume        3713 non-null   int64  
 5   Dividends     3713 non-null   float64
 6   Stock Splits  3713 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 232.1+ KB


In [15]:
btc.index = pd.to_datetime(btc.index)

# del btc['Dividends']
# del btc['Stock Splits']
# Why did I comment those out? Because they were annoying errors!
# I hate it here

btc.columns = [c.lower() for c in btc.columns]

btc.info()
# print(btc.head())

# Ok so let's see if I can reset the index
# Which makes sense because the did that in the first one I tried.
btc_reset = btc.reset_index()

# first_date = btc_reset['Date'][0]

# all_dates = btc_reset['Date']

# filtered_data = btc_reset[btc_reset['Date'] > '2024-01-01']

# print(btc_reset['Date'].dtype)

btc['date'] = btc.index

btc = btc_reset.sort_values('date')

btc.info()
print(btc.head())

# Wow look at all those errors I avoided by commenting out things


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3713 entries, 1970-01-01 00:00:00 to 1970-01-01 00:00:00.000003712
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   level_0       3713 non-null   datetime64[ns]     
 1   index         3713 non-null   datetime64[ns]     
 2   date          3713 non-null   datetime64[ns, UTC]
 3   open          3713 non-null   float64            
 4   high          3713 non-null   float64            
 5   low           3713 non-null   float64            
 6   close         3713 non-null   float64            
 7   volume        3713 non-null   int64              
 8   dividends     3713 non-null   float64            
 9   stock splits  3713 non-null   float64            
dtypes: datetime64[ns, UTC](1), datetime64[ns](2), float64(6), int64(1)
memory usage: 319.1 KB


ValueError: cannot insert level_0, already exists

Not going to lie
I'm ashamed I didn't do that print earlier.
Regardless, easy bug to fix.

In [4]:
btc['open'].fillna(method='ffill', inplace=True)
btc['high'].fillna(method='ffill', inplace=True)
btc['low'].fillna(method='ffill', inplace=True)
btc['close'].fillna(method='ffill', inplace=True)
btc['volume'].fillna(method='ffill', inplace=True)

# Is that the right thing to do with volume?
# Who knows!!! Honestly I'll deal with shitty data at this point

print(btc.head())

                                 open        high         low       close  \
Date                                                                        
2014-09-17 00:00:00+00:00  465.864014  468.174011  452.421997  457.334015   
2014-09-18 00:00:00+00:00  456.859985  456.859985  413.104004  424.440002   
2014-09-19 00:00:00+00:00  424.102997  427.834991  384.532013  394.795990   
2014-09-20 00:00:00+00:00  394.673004  423.295990  389.882996  408.903992   
2014-09-21 00:00:00+00:00  408.084991  412.425995  393.181000  398.821014   

                             volume  
Date                                 
2014-09-17 00:00:00+00:00  21056800  
2014-09-18 00:00:00+00:00  34483200  
2014-09-19 00:00:00+00:00  37919700  
2014-09-20 00:00:00+00:00  36863600  
2014-09-21 00:00:00+00:00  26580100  


/tmp/ipykernel_275007/2328986949.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  btc['open'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_275007/2328986949.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc['open'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_275007/2328986949.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The be

In [ ]:
# ok so issue
# reffering to the date is going to be difficult
# I'm not sure how to do that.

# Why am I not able to get that?
